# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from math import sqrt
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

Функция для вычисления метрик

In [2]:
def metrics(test, predict):
    print(f'MAE: {mean_absolute_error(test, predict)}')
    print(f'MSE: {mean_squared_error(test, predict)}')
    print(f'RMSE: {sqrt(mean_squared_error(test, predict))}')
    print(f'MAPE: {sqrt(mean_absolute_percentage_error(test, predict))}')
    print(f'R^2: {r2_score(test, predict)}')

Функция для заполнения пропусков в выборке

In [3]:
def fill_empty_cell(column_name, df):
    if df.dtypes[column_name] == "float64":
        df[column_name] = df[column_name].fillna(df[column_name].mean())
    elif df.dtypes[column_name] == "int64":
        df[column_name] = df[column_name].fillna(df[column_name].median())
    else:
        df[column_name] = df[column_name].fillna(df[column_name].mode())

Ridge

In [4]:
def create_ridge_with_hyperparameter(X_train, y_train) -> Ridge:
    alpha = {'alpha': np.arange(0.1, 1, 0.1)}
    ridge = GridSearchCV(Ridge(), alpha).fit(X_train, y_train)
    a_ridge = ridge.best_params_['alpha']
    return Ridge(alpha=a_ridge).fit(X_train, y_train)

Lasso

In [5]:
def create_lasso_with_hyperparameter(X_train, y_train) -> Lasso:
    alpha = {'alpha': np.arange(0.2, 1, 0.2)}
    lasso = RandomizedSearchCV(Lasso(), alpha).fit(X_train, y_train)
    a_lasso = lasso.best_params_['alpha']
    return Lasso(alpha=a_lasso).fit(X_train, y_train)

In [6]:
table = pd.read_csv("../data/energy_task.csv")
table.drop(["date"], axis=1, inplace=True)
table.head(5)

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.900000,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.863333,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.2,48.730000,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9


In [7]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Appliances   19735 non-null  int64  
 1   lights       19735 non-null  int64  
 2   T1           19735 non-null  float64
 3   RH_1         19735 non-null  float64
 4   T2           19735 non-null  float64
 5   RH_2         19569 non-null  float64
 6   T3           19735 non-null  float64
 7   RH_3         19735 non-null  float64
 8   T4           19735 non-null  float64
 9   RH_4         19654 non-null  float64
 10  T5           19735 non-null  float64
 11  RH_5         19644 non-null  float64
 12  T6           19735 non-null  float64
 13  RH_6         19651 non-null  float64
 14  T7           19681 non-null  float64
 15  RH_7         19735 non-null  float64
 16  T8           19613 non-null  float64
 17  RH_8         19735 non-null  float64
 18  T9           19651 non-null  float64
 19  RH_9

Проверка датасета на наличие пустых данных

In [8]:
null_columns = []
for i in table.columns:
    if len(table[table[i].isnull()]) > 0:
        print(f"В столбце {i} есть пустые элементы")
        null_columns.append(i)
print(null_columns)

В столбце RH_2 есть пустые элементы
В столбце RH_4 есть пустые элементы
В столбце RH_5 есть пустые элементы
В столбце RH_6 есть пустые элементы
В столбце T7 есть пустые элементы
В столбце T8 есть пустые элементы
В столбце T9 есть пустые элементы
В столбце RH_9 есть пустые элементы
В столбце T_out есть пустые элементы
В столбце Press_mm_hg есть пустые элементы
В столбце RH_out есть пустые элементы
В столбце Visibility есть пустые элементы
['RH_2', 'RH_4', 'RH_5', 'RH_6', 'T7', 'T8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Visibility']


In [9]:
for name in null_columns:
    fill_empty_cell(name, table)

In [10]:
for i in table.columns:
    if len(table[table[i].isnull()]) > 0:
        print(f"В столбце {i} есть пустые элементы")

In [11]:
table.to_csv('../data/energy_task_compleated.csv') 

Создание обучающей выборки

In [12]:
y = table["Appliances"]
X = table.drop(["Appliances"], axis=1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

1. Простая линейная регрессия

In [14]:
lr = LinearRegression().fit(X_train, y_train)
y1_predict = lr.predict(X_test)
metrics(y_test, y1_predict)

MAE: 53.162528692295695
MSE: 8865.29672581138
RMSE: 94.1557046907482
MAPE: 0.7830190568959838
R^2: 0.15924745220821224


2. L1 - Lasso (подбор гиперпараметра + модель)

In [16]:
l = create_lasso_with_hyperparameter(X_train, y_train)
y2_predict = l.predict(X_test)
metrics(y_test, y2_predict)

c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


MAE: 53.127189120214474
MSE: 8869.456689855037
RMSE: 94.17779297613124
MAPE: 0.7827007053794074
R^2: 0.1588529363249206


3. L2 - Ridge (подбор гиперпараметра + модель)

In [17]:
r = create_ridge_with_hyperparameter(X_train, y_train)
y3_predict = r.predict(X_test)
metrics(y_test, y3_predict)

MAE: 53.16239270134859
MSE: 8865.31159630316
RMSE: 94.15578365827115
MAPE: 0.7830180465040615
R^2: 0.1592460419447731


4. Полиномиальная регрессия

In [18]:
table_p = table.drop(columns=["T1", "RH_4", "RH_6", "T7"])
y_p = table_p["Appliances"]
X_p = table_p.drop(["Appliances"], axis=1)
X_p_train, X_p_test, y_p_train, y_p_test = train_test_split(X_p, y_p, test_size=0.5)

In [19]:
p = PolynomialFeatures(2)
X_PolynomialFeatures_train = p.fit_transform(X_p_train)
X_PolynomialFeatures_test = p.fit_transform(X_p_test)

In [20]:
lr2 = LinearRegression().fit(X_PolynomialFeatures_train, y_p_train)
y_PolynomialFeatures_predict = lr2.predict(X_PolynomialFeatures_test)
metrics(y_p_test, y_PolynomialFeatures_predict)

MAE: 50.51593396523815
MSE: 7885.485999367506
RMSE: 88.80025900506995
MAPE: 0.7584983275105639
R^2: 0.23164363465167048


5. Полиномиальная регрессия + Ridge

In [21]:
r_p = create_ridge_with_hyperparameter(X_PolynomialFeatures_train, y_p_train)
y_p_ridge_predict = r_p.predict(X_PolynomialFeatures_test)
metrics(y_p_test, y_p_ridge_predict)

MAE: 50.45435260446966
MSE: 7883.887032435865
RMSE: 88.79125538270007
MAPE: 0.7577557641820821
R^2: 0.23179943689645366


6. Полиномиальная регрессия + Lasso

In [22]:
l_p = create_lasso_with_hyperparameter(X_PolynomialFeatures_train, y_p_train)
y_p_lasso_predict = l_p.predict(X_PolynomialFeatures_test)
metrics(y_p_test, y_p_ridge_predict)

c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.200e+07, tolerance: 8.403e+03
  model = cd_fast.enet_coordinate_descent(
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.232e+07, tolerance: 8.567e+03
  model = cd_fast.enet_coordinate_descent(
c:\Users\bu

MAE: 50.45435260446966
MSE: 7883.887032435865
RMSE: 88.79125538270007
MAPE: 0.7577557641820821
R^2: 0.23179943689645366


c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.042e+07, tolerance: 1.062e+04
  model = cd_fast.enet_coordinate_descent(
